In [10]:
import datetime
import os
import seaborn as sns
import sys
import time

%matplotlib inline

In [11]:
SCRIPTS_DIR = '../app'

sys.path.insert(1, SCRIPTS_DIR)

## Objective: New features to MA Trader 
### [1] Feat: MaxDrawdown as metric and NumExecution for information display [100%]
### [2] Feat: new trading strategy based upon double moving averages [100%]
### [3] Feat: new trading strategy w/ EMA [10%]
### [4] Feat: new trading strategy w/ MACD

#### Prepare Data

In [12]:
# config

SECONDS_IN_ONE_DAY = 86400

#STRATEGIES = ['MA-SELVES', 'DOUBLE-MA']
STRATEGIES = ['DOUBLE-MA']

# simulation configuration
TOL_PCTS = [0.08, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]
BUY_PCTS = [0.3, 0.4, 0.5, 0.6, 0.7]
SELL_PCTS = [0.3, 0.4, 0.5, 0.6, 0.7]

MA_LENGTHS = [3, 5, 10, 20, 30]
EMA_LENGTHS = [10,26]

BUY_STAS = ('by_percentage')
SELL_STAS = ('by_percentage')

# currencies (digital + real-world)
CURS = ['BTC', 'ETH', 'LTC', 'BCH', 'ETC']
FIAT = ['USD']

# signals
NO_ACTION_SIGNAL = 'NO ACTION'
BUY_SIGNAL = 'BUY'
SELL_SIGNAL = 'SELL'

# epilson
EP_COIN = 10e-3
EP_CASH = 5

In [13]:
from trader_driver import TraderDriver
from util import load_csv

name,l = load_csv(csv_fn='../tests/fixtures/BTC_HISTORY.csv')

l = l[-30:]

In [14]:
init_amount = 3000
cur_coin = 5
mode = 'normal'

t_driver = TraderDriver(
    name=name,
    init_amount=init_amount,
    overall_stats=STRATEGIES,
    cur_coin=cur_coin,
    tol_pcts=TOL_PCTS,
    ma_lengths=MA_LENGTHS,
    ema_lengths=EMA_LENGTHS,
    buy_pcts=BUY_PCTS,
    sell_pcts=SELL_PCTS,
    buy_stas=BUY_STAS,
    sell_stas=SELL_STAS,
    mode=mode
)

t_driver.feed_data(data_stream=l)

info = t_driver.best_trader_info
# best trader
best_t = t_driver.traders[info['trader_index']]
# for a new price, find the trade signal
signal = best_t.trade_signal

signal

--> executed=feed_data using time=0.13 seconds.


{'action': 'NO ACTION', 'buy_percentage': 0.7, 'sell_percentage': 0.7}

In [15]:
best_t.all_history[:2]

[{'action': 'BUY',
  'price': 11334.0267420322,
  'date': '2020-10-12',
  'coin': 5.18157712583895,
  'cash': 900.0,
  'portfolio': 59628.133710161004},
 {'action': 'BUY',
  'price': 11666.211700321399,
  'date': '2020-10-13',
  'coin': 5.234499189646774,
  'cash': 270.0,
  'portfolio': 61336.77569158008}]

In [16]:
best_t.all_history_trade_only

[{'action': 'BUY',
  'price': 11334.0267420322,
  'date': '2020-10-12',
  'coin': 5.18157712583895,
  'cash': 900.0,
  'portfolio': 59628.133710161004},
 {'action': 'BUY',
  'price': 11666.211700321399,
  'date': '2020-10-13',
  'coin': 5.234499189646774,
  'cash': 270.0,
  'portfolio': 61336.77569158008},
 {'action': 'BUY',
  'price': 11443.7327918222,
  'date': '2020-10-14',
  'coin': 5.250684467944461,
  'cash': 81.0,
  'portfolio': 60168.43002532753},
 {'action': 'BUY',
  'price': 11395.54736637,
  'date': '2020-10-15',
  'coin': 5.255560582993044,
  'cash': 24.300000000000004,
  'portfolio': 59914.28956032437},
 {'action': 'BUY',
  'price': 11504.5490067109,
  'date': '2020-10-16',
  'coin': 5.2570095576543086,
  'cash': 7.290000000000003,
  'portfolio': 60486.81408478159},
 {'action': 'BUY',
  'price': 11355.160439450901,
  'date': '2020-10-17',
  'coin': 5.25744996886881,
  'cash': 2.187000000000001,
  'portfolio': 59701.37489889149},
 {'action': 'BUY',
  'price': 11358.70637962

In [17]:
best_t.max_drawdown, best_t.num_transaction, best_t.high_strategy

(0.0267, 13, 'DOUBLE-MA')

In [18]:
info

{'buy_pct': 0.7,
 'sell_pct': 0.7,
 'tol_pct': 0.5,
 'buy': 'by_percentage',
 'sell': 'by_percentage',
 'init_value': 59628.133710161004,
 'max_final_value': 68890.479,
 'rate_of_return': '15.534%',
 'baseline_rate_of_return': '14.903%',
 'coin_rate_of_return': '15.607%',
 'trader_index': 249}

In [19]:
best_t.crypto_prices[-10:]

[(11666.211700321399, '2020-10-13'),
 (11443.7327918222, '2020-10-14'),
 (11395.54736637, '2020-10-15'),
 (11504.5490067109, '2020-10-16'),
 (11355.160439450901, '2020-10-17'),
 (11358.706379629699, '2020-10-18'),
 (11471.002547787, '2020-10-19'),
 (11776.592997329499, '2020-10-20'),
 (11936.3629133377, '2020-10-21'),
 (13102.9344615485, '2020-10-22')]

In [20]:
best_t.moving_averages

{'3': 12271.963457405232,
 '5': 11929.11985992648,
 '10': 11701.08006043078,
 '20': 11271.521320013331,
 '30': 11070.433430289284}